In [1]:
import u3
import time
import random
import threading
import csv
import pandas as pd

In [25]:
'''Need to create your own environemnt in order to use the u3 import.
Go to LabJack website and download required packages. https://support.labjack.com/docs/python-for-ljm-windows-mac-linux
Then I create the environment.'''

'Need to create your own environemnt in order to use the u3 import \nI did it using Conda and by just inputting the neccessary Libraries Eddie sent me'

In [2]:
# Constant
MIN_VOLTAGE_OUT = 0.1
MAX_VOLTAGE_OUT = 4.9
MIN_VOLTAGE_IN = 0
MAX_VOLTAGE_IN = 10
MIN_PRESSURE = 0
MAX_PRESSURE = 50
MIN_DEFLATE = 0
MAX_DEFLATE = 25
PRESSURE_TOLERANCE = 0.3  # PSI margin for hitting the target

In [3]:
#Threading Changes
pressure_state = {'TargetPressure': 0}

In [4]:
# Function to close the solenioid valve
def close_solenoid():
    d.setFIOState(4, 1)  # Low signal: Solenoid closed
    print(f"Solenoid Valve: CLOSED")

#Function to  open the solenoid valve
def open_solenoid():
    d.setFIOState(4, 0)  # High signal: Solenoid opened 
    print(f"Solenoid Valve: OPEN")

In [11]:
#Function to map one value to another
def map(value, from_min, from_max, to_min, to_max):
    return (value - from_min) / (from_max - from_min) * (to_max - to_min) + to_min

def get_pressure():
    try:
        #read analog input on AIN2 and convert to volage +-10V
        ainValue = d.getAIN(posChannel=2, negChannel=31, longSettle=False, quickSample=False)
        #convert to pressure
        pressure = map(ainValue, MIN_VOLTAGE_IN, MAX_VOLTAGE_IN, MIN_PRESSURE, MAX_PRESSURE)
        return float(pressure)
    except Exception as e:
        print(f"Error reading AIN2: {e}")
        return None

def set_pressure(pressure):
    try:
        #offset_pressure = offset(pressure)
        regulator_command = map(pressure, MIN_PRESSURE, MAX_PRESSURE, MIN_VOLTAGE_OUT, MAX_VOLTAGE_OUT)
        #converts a voltage to bits to DAC specified
        DAC0_VALUE = d.voltageToDACBits(volts=regulator_command, dacNumber=0, is16Bits=False)
        #dac0 8 bits feedback takes value between 0-255
        d.getFeedback(u3.DAC0_8(DAC0_VALUE))
        #Dictionary that stores pressure value for target pressure
        pressure_state['TargetPressure'] = pressure       
        return pressure
    except Exception as e:
        print(f"Error writing to DAC0: {e}")

# Function to generate random pressure that represents slippage
def random_deflate():
    try:
        #Generates random values for the balloon to be deflated
        value = random.randint(MIN_DEFLATE * 2, (MAX_DEFLATE * 2) - 1)
        #Makes it at .5 intervals
        value = value / 2
        set_pressure(value)
        return value
    except Exception as e:
        print(f"Input Error: {e}")
        return None
        
keep_adjusting = True
#Makes sure pressure stays the same throughout the code
def adjust_pressure(target_pressure, label = "Target"):
    print(f"{label}: Setting pressure to...{target_pressure} ")
    while keep_adjusting:
        current_pressure = get_pressure()
        #Checks to see if the current pressure exceeds target pressure, if it does, adjust
        if abs(current_pressure - target_pressure) > PRESSURE_TOLERANCE:
            print(f"{label}: Pressure drifting ({current_pressure:.2f} PSI) — adjusting...")
            open_solenoid()
            set_pressure(target_pressure)
            time.sleep(3)
        else:
            #print(f"{label}: Pressure stable at {current_pressure:.2f} PSI")
            close_solenoid() #Once pressure is stable, close solenoid
            break

def deflate(threshold=0.5):
    print("Deflation initiated")
    set_pressure(MIN_PRESSURE)
    open_solenoid()
    #Checks if pressure is below 0.3 before closing solenoid
    while get_pressure() >= threshold:
        set_pressure(MIN_PRESSURE)
        current_pressure = get_pressure()
        print(f"Waiting for balloon to deflate. Current pressure: {current_pressure:.2f} PSI")
        time.sleep(3)
    close_solenoid()
    print("Balloon is deflated.")

def emergency_stop():
     print("Emergency stop triggered: Deflating and securing system.")
     deflate()
     print("Emergency stop complete. Solenoid closed and system safe.")
     exit()

In [12]:
df = pd.DataFrame(columns=["Time", "BonePressure", "TargetPressure"])
lock = threading.Lock()
keep_logging = True  # Flag to stop thread

#Saves current pressure, time, and target pressure into a df
def log_bone_data():
    global df
    while keep_logging:
        pressure = get_pressure()
        timestamp = time.strftime("%H:%M:%S")
        with lock: 
            t_pressure = pressure_state['TargetPressure']
            df.loc[len(df)] = [timestamp, pressure, t_pressure]
            time.sleep(1)     

In [13]:
#Converts df to csv
def df_to_csv():
    #Saves the CSV under the timestamp program ends
    file_time = time.strftime("%H%M")
    file_name = f"{file_time}.csv"
    df.to_csv(file_name, index= False)
    print(f"Data saved to: {file_name}")

In [14]:
# Initializes LabJack
try:
    d = u3.U3()
    d.close()
    d = u3.U3()
except Exception as e:
    print(f"Error opening U3 device: {e}")
    exit()

In [16]:
try:
    #Logging thread
    logging_thread = threading.Thread(target=log_bone_data) 
    logging_thread.start()

    #The maximum pressure of the balloon for initialization shouldn't exceed 25 psi
    initial_pressure = 25
    #Time interval between slips
    time_interval = 10  # seconds
    slipping = False   

    open_solenoid()
    set_pressure(initial_pressure)
    adjust_pressure(initial_pressure)
    start_time = time.time()
    
    # Wait for slip initiation
    while not slipping:
        current_time = time.time()
        current_pressure = get_pressure()
        time.sleep(3)
        if (current_time - start_time) >= time_interval:
            user_input = input("Press 's' to tinitiate slip: ").lower().strip()
            if user_input == 's':
                slipping = True
            else:
                print("Waiting for slip initiaion. Maintaining initial pressure...")
                time.sleep(5)  # Wait before prompting again

    # Start slip
    open_solenoid()
    slip_pressure = random_deflate() 
    time.sleep(3)
    slip_start_time = time.time()

    # Maintain slip pressure for 60 seconds
    while (time.time() - slip_start_time) < time_interval:
        current_pressure = get_pressure()
        print(f"Current pressure: {current_pressure}")
        time.sleep(1)
    print("Slip phase completed.")

    # Prompt user to deflate
    while True:
        user_input = input("Press 'd' to initiate deflation: ").lower().strip()
        if user_input == 'd':
            print("Deflating...")
            deflate()
            keep_logging = False
            break
        else:
            print("Maintaining slip pressure...")
            time.sleep(5)
    df_to_csv()        

except KeyboardInterrupt:
    emergency_stop()
    df_to_csv()

except Exception as e: 
    emergency_stop()
    df_to_csv()
    print(f"Error occurred: {e}")    

Solenoid Valve: OPEN
Target: Setting pressure to...25 
Target: Pressure drifting (1.85 PSI) — adjusting...
Solenoid Valve: OPEN
Target: Pressure drifting (20.27 PSI) — adjusting...
Solenoid Valve: OPEN
Target: Pressure drifting (21.30 PSI) — adjusting...
Solenoid Valve: OPEN
Target: Pressure drifting (23.21 PSI) — adjusting...
Solenoid Valve: OPEN
Target: Pressure drifting (24.41 PSI) — adjusting...
Solenoid Valve: OPEN
Target: Pressure drifting (24.56 PSI) — adjusting...
Solenoid Valve: OPEN
Target: Pressure drifting (24.66 PSI) — adjusting...
Solenoid Valve: OPEN
Target: Pressure drifting (24.66 PSI) — adjusting...
Solenoid Valve: OPEN
Solenoid Valve: CLOSED


Press 's' to tinitiate slip:  s


Solenoid Valve: OPEN
Current pressure: 14.439999999999992
Current pressure: 13.158879999999993
Current pressure: 12.204319999999997
Current pressure: 11.526079999999999
Current pressure: 11.099039999999993
Current pressure: 11.07391999999999
Current pressure: 11.400479999999993
Current pressure: 10.646879999999994
Current pressure: 3.839359999999994
Current pressure: 3.713759999999997
Slip phase completed.


Press 'd' to initiate deflation:  d


Deflating...
Deflation initiated
Solenoid Valve: OPEN
Waiting for balloon to deflate. Current pressure: 3.14 PSI
Solenoid Valve: CLOSED
Balloon is deflated.
Data saved to: 1407.csv
